In [199]:
import numpy as np
import imageio
from PIL import Image

import math
from math import fabs

In [218]:
ilosc_katow = 10 #kolumny
ilosc_emiterow = 50 #wiersze
l = 350 #odleglosc od emitera nr 1 do emitera nr ilosc_emiterow

### Wczytanie zdjęcia

In [219]:
image = imageio.imread('Zdjecia/SADDLE_PE.JPG')

### Rozmiar zdjęcia

In [220]:
image.shape

(876, 1053, 3)

### Zapisanie wysokości i szerokości zdjęcia

In [221]:
n_x = len(image)
n_y = len(image[0])

n_x, n_y

(876, 1053)

### Ogólny algorytm Bresenhama 

### Zapalanie piksela - rysowanie lini

In [222]:
def zapalPiksel(x,y, kolor="czarny"):
    if kolor=="czarny":
        wartosc = 255
    elif kolor=="bialy":
        wartosc=0
        
    #image_m[x][y] = wartosc
    image[x][y] = [wartosc, wartosc, wartosc]

In [240]:
def zczytajPiksel(nr_emitera, nr_kata, x, y):
    try:
        sinog[nr_emitera][nr_kata] += image[x][y][0]
    except:
        pass

### Algorytm

In [296]:
### Ogólny algorytm Bresenhama
###############
def algBresenhama(nr_emitera, nr_kata, x1,y1,x2,y2):
#K01: 	Jeżeli x1 ≤ x2, to kx ← 1, inaczej kx ← -1 	; określamy krok X od x1 do x2
    if(x1<=x2):
        kx = 1
    else:
        kx = -1
#K02: 	Jeżeli y1 ≤ y2, to ky ← 1, inaczej ky ← -1 	; określamy krok Y od y1 do y2
    if(y1<=y2):
        ky = 1
    else:
        ky = -1
#K03: 	dx ← |x2 - x1| 	; odległość pomiędzy x1 i x2
    dx = math.fabs(x2 - x1)
#K04: 	dy ← |y2 - y1| 	; odległość pomiędzy y1 i y2
    dy = math.fabs(y2 - y1)
#K05: 	Zapal piksel x1,y1 	; pierwszy piksel odcinka
    #zapalPiksel(x1, y1)
#K06: 	Jeżeli dx < dy, idź do kroku K16 	; dla kątów > 45° wykonujemy wersję algorytmu z przestawionymi współrzędnymi
    if(dx < dy):
        #K16: 	e ← dy / 2 	; wersja algorytmu Bresenhama ze zamienionymi współrzędnymi x i y
        e = dy/2
        for i in range(int(dy)):
#K17: 	Powtarzaj dy razy kroki K18...K23 
#K18: 	    y1 ← y1 + ky
            y1 = y1 + ky
#K19: 	    e ← e - dx 
            e = e - dx
#K20: 	    Jeżeli e ≥ 0, idź do kroku K23 	 
            if(e>=0):
                try:
                    l_op_sinog[nr_emitera][nr_kata] += 1
                    zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                except:
                    pass
                #zapalPiksel(x1, y1)
            else:
#K21: 	    x1 ← x1 + kx 	 
                x1 = x1 + kx
#K22: 	    e ← e + dy 	 
                e = e + dy
#K23: 	    Zapal piksel x1,y1 	 
                try:
                    l_op_sinog[nr_emitera][nr_kata] += 1
                    zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                except:
                    pass
                #zapalPiksel(x1, y2)
    else:
#K07: 	e ← dx / 2 	; obliczamy wartość początkową wyrażenia błędu
        e = (dx/2)
#K08: 	Powtarzaj dx razy kroki K09...K14 	; rysujemy pozostałe piksele w pętli
        for i in range(int(dx)):
#K09: 	    x1 ← x1 + kx 	; przesuwamy się w odpowiednią stronę w kierunku szybkim
            x1 = x1 + kx
#K10: 	    e ← e - dy 	; obliczamy wyrażenie błędu
            e = e - dy
#K11: 	    Jeżeli e ≥ 0, idź do kroku K14 	; jeśli wyrażenie błędu jest nieujemne, pomijamy ruch w kierunku wolnym
            if(e>0):
                try:
                    l_op_sinog[nr_emitera][nr_kata] += 1
                    zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                except:
                    pass
                #zapalPiksel(x1, y1)
            else:
#K12: 	    y1 ← y1 + ky 	; przesuwamy się w odpowiednią stronę w kierunku wolnym
                y1 = y1 + ky
#K13: 	    e ← e + dx 	; obliczamy nowe wyrażenie błędu
                e = e + dx
#K14: 	    Zapal piksel x1.y1 	; kolejny piksel odcinka
                try:
                    l_op_sinog[nr_emitera][nr_kata] += 1
                    zczytajPiksel(nr_emitera, nr_kata, x1,y1)
                except:
                    pass
                #zapalPiksel(x1, y1)

### Zapisanie obrazu z tablicy do wersji graficznej .png, oraz wyświetlenie

In [297]:
# sinogram_array = np.asarray(sinogram)
def arrayToImage(arr, form='L'):
    img = Image.fromarray(arr, form)
    img.save('my.png')
    img.show()

### Sinogram

In [298]:
# suma pikseli dla kazdego [emiter][kat]
sinog = [ [ 0 for y in range( ilosc_katow ) ] for x in range( ilosc_emiterow ) ]
# ilosc skladnikow odpowiadajaca kazdej komorce z tablicy sinog
l_op_sinog = [ [ 0 for y in range( ilosc_katow ) ] for x in range( ilosc_emiterow ) ]
# wartosc srednia dla kazdego [emiter][kat] -> sinog/l_op_sinog
sinogram = [ [ 0 for y in range( ilosc_katow ) ] for x in range( ilosc_emiterow ) ]

In [299]:
len(sinog), len(sinog[0])

(50, 10)

In [300]:
# polozenie emiterow odbiorczych

x_em = [ None for x in range(ilosc_emiterow + 1)]
y_em = [ None for y in range(ilosc_emiterow + 1)]

def ustaw_polozenie_emiterow_odb(bok):
    odstep = int(n_y/(ilosc_emiterow-1))
    dod = 0
    if bok==0:
        for i in range(1, ilosc_emiterow+1):
            x_em[i] = n_y - 1
            y_em[i] = 0 + dod
            dod += odstep
    elif bok==1:
        for i in range(1, ilosc_emiterow+1):
            x_em[i] = 0 + dod
            y_em[i] = 0
            dod += odstep
    elif bok==2:
        for i in range(1, ilosc_emiterow+1):
            x_em[i] = 0
            y_em[i] = 0 + dod
            dod += odstep
    elif bok==3:
        for i in range(1, ilosc_emiterow+1):
            x_em[i] = 0 + dod
            y_em[i] = n_x - 1
            dod += odstep

In [301]:
def ustaw_emiter_glowny(bok, przesuniecie=0):
#położenie początkowe emitera głównego
    if bok==0:
        y_em[0] = przesuniecie
        x_em[0] = 0
    elif bok==1:
        y_em[0] = n_y-1
        x_em[0] = przesuniecie
    elif bok==2:
        y_em[0] = przesuniecie
        x_em[0] = n_x -1
    elif bok==3:
        y_em[0] = 0
        x_em[0] = przesuniecie

In [302]:
def norma_res_to_sinogram():
    for k in range(ilosc_katow):
        for e in range(ilosc_emiterow):
            try:
                val = int(round(sinog[e][k]/l_op_sinog[e][k]))
            except:
                val = 0
            sinogram[e][k] = val

In [303]:
#for i in range(ilosc_katow):
 #   ustaw_emiter_glowny(i)
  #  for em in range(ilosc_emiterow):
   #     algBresenhama(em, i, x_em[0], y_em[0], x_em[em+1], y_em[em+1])
    #norma_res_to_sinogram()

In [304]:
for bok in range(4):
    ustaw_polozenie_emiterow_odb(bok)
    ods = 0
    przes = int(n_y/ilosc_katow)
    for i in range(ilosc_katow):
        ustaw_emiter_glowny(bok, ods)
        for em in range(ilosc_emiterow):
            algBresenhama(em, i, x_em[0], y_em[0], x_em[em+1], y_em[em+1])
        ods += przes
norma_res_to_sinogram()
        

sinogram_array = np.asarray(sinogram)
arrayToImage(sinogram_array, form="L")

In [305]:
sinogram_array = np.asarray(sinogram)
sinogram_array.shape
sinogram_array

array([[ 24,  56,  53,  29,  35,  71,  74,  79,  73,  61],
       [ 24,  70,  45,  29,  39,  71,  78,  79,  69,  65],
       [ 25,  82,  40,  28,  42,  74,  81,  79,  69,  63],
       [ 28,  71,  38,  31,  47,  76,  82,  82,  69,  63],
       [ 40,  70,  39,  35,  51,  77,  83,  83,  68,  66],
       [ 55,  60,  37,  37,  58,  77,  83,  82,  69,  67],
       [ 74,  55,  32,  40,  61,  72,  83,  77,  69,  66],
       [ 74,  53,  33,  44,  61,  70,  85,  77,  74,  68],
       [ 68,  46,  30,  46,  64,  74,  89,  77,  74,  71],
       [ 54,  45,  35,  48,  65,  74,  87,  77,  74,  75],
       [ 43,  41,  37,  52,  65,  77,  87,  78,  75,  80],
       [ 42,  38,  42,  57,  64,  81,  88,  77,  74,  83],
       [ 38,  34,  45,  59,  67,  81,  89,  79,  76,  84],
       [ 33,  38,  47,  60,  67,  82,  85,  81,  73,  86],
       [ 30,  40,  47,  59,  73,  85,  82,  82,  76,  86],
       [ 30,  46,  48,  55,  78,  92,  84,  80,  83,  80],
       [ 33,  50,  52,  54,  84,  96,  83,  79,  83,  76

In [151]:
arrayToImage(sinogram_array)

In [146]:
x_em

[0,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399,
 399]

In [147]:
y_em

[396,
 0,
 4,
 8,
 12,
 16,
 20,
 24,
 28,
 32,
 36,
 40,
 44,
 48,
 52,
 56,
 60,
 64,
 68,
 72,
 76,
 80,
 84,
 88,
 92,
 96,
 100,
 104,
 108,
 112,
 116,
 120,
 124,
 128,
 132,
 136,
 140,
 144,
 148,
 152,
 156,
 160,
 164,
 168,
 172,
 176,
 180,
 184,
 188,
 192,
 196,
 200,
 204,
 208,
 212,
 216,
 220,
 224,
 228,
 232,
 236,
 240,
 244,
 248,
 252,
 256,
 260,
 264,
 268,
 272,
 276,
 280,
 284,
 288,
 292,
 296,
 300,
 304,
 308,
 312,
 316,
 320,
 324,
 328,
 332,
 336,
 340,
 344,
 348,
 352,
 356,
 360,
 364,
 368,
 372,
 376,
 380,
 384,
 388,
 392,
 396]

In [149]:
l_op_sinog[1][3]

798